Converts the master CSV (breaks down years by institution, sex, and race/ethnicity for grad students and postdoctorates) into two tree-based dictionaries: one for grad students and one for postdocs. Each dictionary is pickled.

In [1]:
import pickle
import pandas as pd

In [2]:
# Converts a dataframe into a nested dictionary
# https://stackoverflow.com/questions/19798112/convert-pandas-dataframe-to-a-nested-dict
def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.ix[:,1:]) for k,g in grouped}
    return d

In [45]:
df = pd.read_csv(
    '../data/Detailed_Sex_RaceEthnicity.csv', 
    thousands=',',
    header=6, # starting at 6th row
    skipfooter=6, # ignore last 4 lines of data (footer)
    engine='python', # default pandas engine does not support skipfooter
    index_col=False, # don't use first column (year) as dataframe index
    names = ['Year', 'Institution Name', 'Detailed Fields', 'Sex', 'Race and Ethnicity', 'Postdoctorates', 'Graduate Students']
)

df['Postdoctorates'] = pd.to_numeric(df['Postdoctorates'])

# Delete some aggregate values we don't need for this analysis
df = df[ df['Institution Name'] != 'Total for selected values' ]
df = df[ df['Detailed Fields'] != 'Total for selected values' ]

# Rename all instances of 'Columbia Univ in the City of New York'
# to 'Columbia University in the City of New York'
df['Institution Name'] = df['Institution Name'].str.replace(
    'Columbia Univ in the City of New York', 
    'Columbia University in the City of New York'
)

df.tail()

,Year,Institution Name,Detailed Fields,Sex,Race and Ethnicity,Postdoctorates,Graduate Students
105540,1994,Yale University,Surgery,Female,Ethnicity/race not collected for 2009 and prior,4,0
105541,1994,Yale University,Surgery,Female,"Foreign nationals holding temporary visas, reg...",5,0
105542,1994,Yale University,Surgery,Male,Total for selected values,41,0
105543,1994,Yale University,Surgery,Male,Ethnicity/race not collected for 2009 and prior,7,0
105544,1994,Yale University,Surgery,Male,"Foreign nationals holding temporary visas, reg...",34,0


In [ ]:
postdocs = df.drop('Graduate Students', 1)
postdocs = postdocs[postdocs['Postdoctorates'] > 0]

with open('../data/Detailed_Sex_RaceEthnicity_Postdoctorates.p', 'wb') as f:
    pickle.dump(recur_dictify(postdocs), f)

/Users/jasonkao/.local/share/virtualenvs/international-students-qEUJUUUW/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [ ]:
grads = df.drop('Postdoctorates', 1)
grads = grads[grads['Graduate Students'] > 0]

with open('../data/Detailed_Sex_RaceEthnicity_Graduates.p', 'wb') as f:
    pickle.dump(recur_dictify(grads), f)